# Collect data to train a model, save as RGB

In [ ]:
import mss
import cv2
import numpy as np
import time
import os
import keyboard


save_folder = 'captured_screens'
os.makedirs(save_folder, exist_ok=True)


counter = 1

with mss.mss() as sct:
   

    while True:
        
        if keyboard.is_pressed('q'):
            print("Durduruldu.")
            break

        
        screen = np.array(sct.grab(gameScreen))

        

        filename = f"{counter}.png"
        file_path = os.path.join(save_folder, filename)

        
        cv2.imwrite(file_path, screen)

        counter += 1
        
        
        time.sleep(1)

# Run the game bot.

In [ ]:
import numpy as np
import cv2
import mouse
from ultralytics import YOLO
import time
import pygetwindow as gw
import keyboard
import win32gui, win32ui, win32con

class WindowCapture:
    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[..., :3]
        img = np.ascontiguousarray(img)
            
        return img

model = YOLO('best.pt')

def slice(x, y, height):
    mouse.move(int(x), int(y), absolute=True)
    mouse.drag(0, 0, 0, -(int(height * 2.25)), absolute=False, duration=0.025)

game_window = gw.getWindowsWithTitle('MuMu Player 12')[0]
wincap = WindowCapture('MuMu Player 12')  
slice_count = 0  

while True:  
    screen = wincap.get_screenshot()
    screen_rgb = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)

    
    results = model.track(screen_rgb, conf=0.7, iou=0.4)

    for result in results:
        boxes = result.boxes.xyxy  
        classes = result.boxes.cls  
        scores = result.boxes.conf
        
        for box, clas in zip(boxes, classes):
            if clas == 1:  
                x_min, y_min, x_max, y_max = box
                center_x = (x_min + x_max) / 2
                center_y = (y_min + y_max) / 2
                height = y_max - y_min

                slice(center_x, center_y, height)
                slice_count += 1  
    
        # for box, score in zip(boxes, scores):
        #     if score > 0.5:  
        #         x_min, y_min, x_max, y_max = map(int, box)
        #         cv2.rectangle(screen, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        #         label = f'Score: {score:.2f}'
        #         cv2.putText(screen, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # cv2.imshow('Tespitler', screen)
    # time.sleep(0.1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    if keyboard.is_pressed('q'):
        print("Döngüden çıkılıyor...")
        break

cv2.destroyAllWindows()